In [ ]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np

import pyttsx3
import time
import threading
last_spoken = {} # 같은 객체가 너무 자주 안내되지 않도록 각 객체가 마지막으로 안내된 시간을 저장, ex) {track_id : 시간}
Speak_Interval = 3 # 동일한 객체에 대해서는 3초 이내에 안내를 반복하지 않도록 시간 설정 변수(초 단위)

# 이전에 gTTS API 썼을 때, 너무 자주 요청하니 차단먹어서 이번엔 로컬 음성생성 라이브러리를 씀.======
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()
    engine.stop()
# =================================================================================================

model = YOLO('yolov8s.pt')
tracker = DeepSort(max_age=10)

cap = cv2.VideoCapture('SeoulWalk2_720.mp4')
track_paths = {}
color_map = {}

# ==========바운딩박스 교차 비교 확인 함수 =========
def is_intersect(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    return not (
        x1_max < x2_min or
        x2_max < x1_min or
        y1_max < y2_min or
        y2_max < y1_min
    )
# =============================================

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    results = model(frame, verbose=False)[0]
    #frame = results.plot()

    detections = []
    
    # =======영상 프레임 크기 기준 사용자 근접 영역 계산 =========
    frame_height, frame_width = frame.shape[:2]
    user_x = frame_width//2
    user_y = frame_height - 10
    near_left = frame_width //4
    near_right = frame_width * 3 //4
    near_top = frame_height *3 //4
    user_area = (near_left, near_top, near_right, frame_height)
    # =====================================================
    
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) //2
        
        if track_id not in color_map:
            color_map[track_id] = tuple(np.random.randint(0, 255, size=3).tolist())
        color = color_map[track_id]
        
        class_name = track.get_det_class()
        if class_name == "person":
            class_name_ko="사람이"
        elif class_name == "truck":
            class_name_ko="트럭이"
        elif class_name == "car":
            class_name_ko="자동차가"
        elif class_name == "bus":
            class_name_ko="버스가"
        elif class_name == "bicycle":
            class_name_ko="자전거가"
        elif class_name == "motocycle":
            class_name_ko="오토바이가"

        # 객체 박스 좌표를 변수에 저장하고, 사용자 근접 영역과 겹치는지 확인 -> 겹치는 객체만 박스 표시
        object_box = (x1, y1, x2, y2)
        if is_intersect(object_box, user_area):                    
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f'ID {track_id} {class_name}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        # =============================================================================================
            # == 접근 방향 판단 및 음성 안내 문장 생성 ===========
            direction = ""
            arrow_end = (cx, cy)
            if cx < near_left:
                direction = "왼쪽에서 접근"
                arrow_end = (user_x - 100, user_y)
            elif cx > near_right:
                direction = "오른쪽에서 접근"
                arrow_end = (user_x + 100, user_y)
            elif cy < near_top:
                direction = "앞에서 접근"
                arrow_end = (user_x, user_y - 100)
            else:
                direction = "가까이서 접근"
            cv2.arrowedLine(frame, (user_x, user_y), arrow_end, color, 3, tipLength=0.3)
            # ====================================================

            # ==================== 빠른 접근 경고 ===========================================================
            if track_id in track_paths and len(track_paths[track_id]) >= 2:
                prev_cx, prev_cy = track_paths[track_id][-2]
                delta_y = cy - prev_cy
                if delta_y >10:
                    direction = "빠르게 접근"
                    arrow_end = (user_x, user_y - 100)
                    cv2.arrowedLine(frame, (user_x, user_y), arrow_end, (0,0,255), 3, tipLength=0.3)
                    current_time = time.time()
                    if track_id not in last_spoken or current_time - last_spoken[track_id] > Speak_Interval:
                        warning_msg = f"{class_name_ko} 빠르게 접근 중입니다"
                        threading.Thread(target=speak, args=(warning_msg,), daemon=True).start()
                        last_spoken[track_id] = current_time
                    continue
            # ================================================================================================

            
            # 현재 시간 저장하고, 안내한적 없거나 인터벌 시간이 지난 객체를 근접 객체로 음성 안내함
            current_time = time.time()
            if track_id not in last_spoken or current_time - last_spoken[track_id] > Speak_Interval:
                msg = f"{class_name_ko} {direction} 중입니다"
                threading.Thread(target=speak, args=(msg,), daemon=True).start()
                last_spoken[track_id] = current_time
            # =====================================================================================
                
        # 추적되지 않은 새로운 객체일 경우 track_paths에 id를 등록하고 추적선 그릴 좌표를 저장
        if track_id not in track_paths:
            track_paths[track_id] = []
        track_paths[track_id].append((cx, cy))
        
        # 객체 이동 경로를 선으로 표시하는 코드
        path = track_paths[track_id]
        for i in range(1, len(path)):
            cv2.line(frame, path[i-1], path[i], color, 2)

    # 사용자를 작은 원으로 표시하고 사용자 근접 영역을 박스로 표시
    cv2.circle(frame, (user_x, user_y), 10, (0,0,255), -1)
    cv2.putText(frame, 'User', (user_x-10, user_y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255),2)
    cv2.rectangle(frame, (near_left, near_top), (near_right, frame_height), (255,0,0), 2)
    
    cv2.imshow("YOLOv8 plot + Deep Sort Tracking Visualization", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()